In [1]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

key = os.environ['av_key']

In [2]:
cc = CryptoCurrencies(key=key, output_format='pandas')

In [3]:
df, meta_data = cc.get_crypto_intraday('ETH', market='USD', interval='1min', outputsize='full')

In [4]:
df = df.rename(columns={"4. close": "close",
                   "1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "5. volume": "volume"})

df = df[['open', 'high', 'low', 'close', 'volume']]

In [5]:
df = df.sort_index()
df['idx_int'] = np.arange(0, len(df))
df = df.reset_index()

In [6]:
df.head()

,date,open,high,low,close,volume,idx_int
0,2024-11-09 13:39:00,3043.92,3048.80,3043.92,3047.17,82.0,0
1,2024-11-09 13:40:00,3047.18,3049.97,3047.18,3048.91,103.0,1
2,2024-11-09 13:41:00,3049.39,3049.55,3046.63,3049.48,70.0,2
3,2024-11-09 13:42:00,3049.50,3049.75,3047.14,3048.71,53.0,3
4,2024-11-09 13:43:00,3048.64,3048.80,3046.86,3046.96,33.0,4


In [7]:
def compute_gradient(start_index, df, x_label, y_label, len_data):
    # Ensure we only take data points from n to n+5
    if start_index + len_data > len(df):
        return None  # Return None if there are not enough points to calculate gradient
    
    # Extract the last 5 data points (x, y) from the DataFrame
    data_segment = df.iloc[start_index:start_index + len_data]
    x = data_segment[x_label]
    y = data_segment[y_label]
    
    # Calculate the necessary summations for the least squares formula
    n = len(x)
    sum_x = x.sum()
    sum_y = y.sum()
    sum_x2 = (x**2).sum()
    sum_xy = (x * y).sum()
    
    # Calculate the slope (gradient) using the least squares formula
    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x**2)
    return slope

def check_crossing(df, col1, col2):
    # Calculate the difference between the two columns
    diff = df[col1] - df[col2]
    diff = diff/np.abs(diff)
    # Check if there is a sign change in the difference
    crossing = ((diff.shift(1) * diff) - 1)/-2
    
    return crossing

In [8]:
# realtime stock trading
df.ta.ema(length=10, append=True)
df.ta.ema(length=50, append=True)
df.ta.rsi(length=14, append=True)
df.ta.bbands(length=20, std=2, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df.ta.psar(append=True)
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
345,35.957506,29.365688,17.901111
346,34.607984,28.465423,20.166735
347,33.586104,29.215466,19.354881
348,32.563624,28.490307,19.283611


In [9]:
try: df = df.rename(columns={'EMA_60':'EMA_50'})
except: print("skip")

In [10]:
df['psar_flip_dir'] = 0
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARl_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = 1
df.loc[(df['PSARr_0.02_0.2']==1) & (df['PSARs_0.02_0.2'].isnull()==False), 'psar_flip_dir'] = -1

In [11]:
df.head()

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir
0,2024-11-09 13:39:00,3043.92,3048.80,3043.92,3047.17,82.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0,NaN,NaN,NaN,0
1,2024-11-09 13:40:00,3047.18,3049.97,3047.18,3048.91,103.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3043.92000,NaN,0.04,0,NaN,NaN,NaN,0
2,2024-11-09 13:41:00,3049.39,3049.55,3046.63,3049.48,70.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3043.92000,NaN,0.04,0,NaN,NaN,NaN,0
3,2024-11-09 13:42:00,3049.50,3049.75,3047.14,3048.71,53.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3044.16200,NaN,0.04,0,NaN,NaN,NaN,0
4,2024-11-09 13:43:00,3048.64,3048.80,3046.86,3046.96,33.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3044.39432,NaN,0.04,0,NaN,NaN,NaN,0


In [12]:
df['r_ema_s_m'] = df['EMA_10'] / df['EMA_50']

In [13]:
len_data = 3

# Loop through the DataFrame, compute gradient for each row (starting point)
for i in range(len(df) - len_data):  # Make sure we have at least 5 points for each calculation
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_10', len_data)
    df.at[i + len_data-1, 'gradient_ema_10'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'RSI_14', len_data)
    df.at[i + len_data-1, 'gradient_rsi_14'] = gradient  # Store the gradient in the row corresponding to n+4
    gradient = compute_gradient(i, df, 'idx_int', 'close', len_data)
    df.at[i + len_data-1, 'gradient_ls'] = gradient
    gradient = compute_gradient(i, df, 'idx_int', 'EMA_50', len_data)
    df.at[i + len_data-1, 'gradient_ema_50'] = gradient  # Store the gradient in the row corresponding to n+4

In [14]:
# mask_crossing = (df['r_ema_s_m']<1.0005)&(df['r_ema_s_m']>0.9995)
# df['flag_ema_crossing'] = 0
# df.loc[mask_crossing, 'flag_ema_crossing'] = 1

In [15]:
df['flag_ema_crossing'] = check_crossing(df, 'EMA_10', 'EMA_50')

In [16]:
df.iloc[-20:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing
330,2024-11-09 19:09:00,3070.58,3073.13,3069.25,3072.82,48.0,330,3070.454964,3053.597614,65.628953,3032.099156,3060.6110,3089.122844,1.863147,0.714104,7.725680,0.909024,6.816656,3066.828063,NaN,0.10,0,42.181595,34.761251,14.445436,0,1.005520,0.327293,-3.260041,-1.320,0.753456,-0.0
331,2024-11-09 19:10:00,3072.69,3072.69,3069.11,3069.77,54.0,331,3070.330425,3054.231825,60.760690,3034.075901,3061.8675,3089.659099,1.815337,0.642174,7.209621,0.314372,6.895249,3068.758257,NaN,0.10,0,42.078130,33.065153,13.924727,0,1.005271,0.200512,-1.407937,-0.370,0.709399,-0.0
332,2024-11-09 19:11:00,3069.65,3072.63,3068.00,3072.02,101.0,332,3070.637620,3054.929401,62.944422,3036.293395,3063.2375,3090.181605,1.759191,0.662976,6.902627,0.005903,6.896724,NaN,3086.130000,0.02,1,41.656623,31.030300,14.543171,-1,1.005142,0.091328,-1.342265,-0.400,0.665893,-0.0
333,2024-11-09 19:12:00,3071.94,3073.59,3071.58,3072.92,35.0,333,3071.052599,3055.634915,63.811959,3038.309830,3064.5210,3090.732170,1.710621,0.660218,6.655238,-0.193189,6.848427,NaN,3085.767400,0.02,0,41.398659,31.498210,14.136445,0,1.005046,0.361087,1.525635,1.575,0.701545,-0.0
334,2024-11-09 19:13:00,3072.90,3079.16,3072.36,3077.67,75.0,334,3072.255762,3056.499036,68.061868,3040.394783,3066.0165,3091.638217,1.671336,0.727415,6.764488,-0.067151,6.831639,NaN,3085.412052,0.02,0,41.843341,36.159962,12.829245,0,1.005155,0.809071,2.558723,2.825,0.784817,-0.0
335,2024-11-09 19:14:00,3078.13,3078.31,3075.13,3075.77,42.0,335,3072.894715,3057.254760,64.784463,3042.808148,3067.3945,3091.980852,1.603077,0.670328,6.621428,-0.168169,6.789597,NaN,3085.063811,0.02,0,42.256259,34.550931,12.258374,0,1.005116,0.921058,0.486252,1.425,0.809923,-0.0
336,2024-11-09 19:15:00,3075.78,3082.24,3071.01,3071.82,384.0,336,3072.699312,3057.825946,58.479822,3045.998561,3068.6655,3091.332439,1.477316,0.569584,6.118787,-0.536648,6.655435,NaN,3084.722535,0.02,0,41.404878,29.550192,15.794122,0,1.004864,0.221775,-4.791023,-2.925,0.663455,-0.0
337,2024-11-09 19:16:00,3071.72,3077.30,3064.25,3077.30,506.0,337,3073.535801,3058.589634,63.750428,3049.273091,3070.1290,3090.984909,1.358634,0.671918,6.092400,-0.450428,6.542828,NaN,3084.388084,0.04,0,39.018195,25.018582,21.315849,0,1.004887,0.320543,-0.517018,0.765,0.667437,-0.0
338,2024-11-09 19:17:00,3078.13,3082.96,3076.28,3079.76,162.0,338,3074.667473,3059.419844,65.846362,3053.249670,3071.7365,3090.223330,1.203673,0.717006,6.198538,-0.275432,6.473970,NaN,3083.582561,0.04,0,37.681929,29.672944,19.654344,0,1.004984,0.984081,3.683270,3.970,0.796949,-0.0
339,2024-11-09 19:18:00,3079.76,3091.51,3079.29,3088.88,383.0,339,3077.251569,3060.575145,72.251861,3056.874339,3073.7180,3090.561661,1.095980,0.950080,6.938576,0.371685,6.566891,3064.250000,NaN,0.02,1,37.459181,35.036859,17.037990,1,1.005449,1.857884,4.250716,5.790,0.992755,-0.0


In [17]:
mask_ema_grad_pos = (df['gradient_ema_10']>0.05)
mask_ema_grad_neg = (df['gradient_ema_10']<-0.05)
df['flag_grad_ema'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema'] = -1

mask_ema_grad_pos = (df['gradient_ema_50']>0.05)
mask_ema_grad_neg = (df['gradient_ema_50']<-0.05)
df['flag_grad_ema_50'] = 0
df.loc[mask_ema_grad_pos, 'flag_grad_ema_50'] = 1
df.loc[mask_ema_grad_neg, 'flag_grad_ema_50'] = -1

mask_rsi_grad_pos = (df['gradient_rsi_14']>=1)
mask_rsi_grad_neg = (df['gradient_rsi_14']<=1)
df['flag_grad_rsi'] = 0
df.loc[mask_rsi_grad_pos, 'flag_grad_rsi'] = 1
df.loc[mask_rsi_grad_neg, 'flag_grad_rsi'] = -1

df['flag_grad_ls'] = 0
df.loc[df['gradient_ls'] >= 0.05, 'flag_grad_ls'] = 1
df.loc[df['gradient_ls'] <= -0.05, 'flag_grad_ls'] = -1

In [18]:
df['ema_short_above_or_below'] = 0
df.loc[(df['EMA_10']>df['EMA_50']), 'ema_short_above_or_below'] = 1
df.loc[(df['EMA_10']<df['EMA_50']), 'ema_short_above_or_below'] = -1

In [19]:
df['r_close_bbu'] = df['close'] / df['BBU_20_2.0']
df['r_close_bbl'] = df['close'] / df['BBL_20_2.0']
df['r_ema_bbu'] = df['EMA_10'] / df['BBU_20_2.0']
df['r_ema_bbl'] = df['EMA_10'] / df['BBL_20_2.0']

# Strategy

To create a real-time trading strategy with specific technical indicator settings and strategies for entering and exiting trades, let's break down the components clearly:

### 1. **Technical Indicators**
Here's how we can set up the specific technical indicators:

#### Moving Averages
- **Short-Term Moving Average (MA)**: Use a 10-period MA. This is effective for spotting short-term trends and suitable for minute-by-minute trading data.
- **Long-Term Moving Average**: Use a 50-period MA to confirm the overall trend direction. This helps filter out the noise and provides a clearer direction.

#### Relative Strength Index (RSI)
- **Length**: A 14-period RSI is standard and works well in a variety of market conditions.
- **Overbought Threshold**: Set at 70 to indicate potential selling points.
- **Oversold Threshold**: Set at 30 to indicate potential buying points.

#### MACD (Moving Average Convergence Divergence)
- **Fast Line**: 12-period Exponential Moving Average (EMA)
- **Slow Line**: 26-period EMA
- **Signal Line**: 9-period EMA of the MACD line itself

### 2. **Entry and Exit Points**
- **Go Long (Buy)**: 
  - **Entry Point**: When the 10-period MA crosses above the 50-period MA and the RSI is below 70 but above 30, indicating neither overbought nor oversold conditions. Additionally, the MACD line should cross above the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back below the 50-period MA, or RSI approaches 70, indicating a potential overbought condition.
  
- **Go Short (Sell)**: 
  - **Entry Point**: When the 10-period MA crosses below the 50-period MA and the RSI is above 30 but below 70, indicating normal conditions. Also, the MACD line should cross below the signal line.
  - **Exit Point (Take Profit)**: Consider exiting when the 10-period MA crosses back above the 50-period MA, or RSI approaches 30, indicating a potential oversold condition.

### 3. **Stop Loss Criteria**
- **For Long Positions**: Place a stop loss below the most recent significant low or a fixed percentage, such as 2-3% below the entry price, depending on your risk tolerance.
- **For Short Positions**: Place a stop loss above the most recent significant high or a fixed percentage, such as 2-3% above the entry price.

### 4. **Risk Management**
- Consider the amount of capital at risk per trade, commonly 1-2% of your total trading capital to manage risk effectively.
- Use trailing stops to lock in profits as the market moves in your favor.

### 5. **Automation and Monitoring**
- Automate the strategy using a trading bot if possible, especially to handle the high-frequency nature of minute-by-minute trading.
- Regularly monitor performance and adjust parameters as necessary based on market conditions and strategy performance.

This setup gives you a robust framework for trading on a minute-by-minute basis, leveraging these technical indicators to make informed trading decisions. Adjust these settings based on backtesting results and your specific risk tolerance.

# Buy Signal

In [20]:
mask_le1 = (df['ema_short_above_or_below']==1)&(df['flag_ema_crossing']==1)&(df['flag_grad_ema']>0)
mask_le2 = (df['MACDh_12_26_9']>0)
mask_le3 = (df['r_close_bbl']<=1.0005)
mask_le4 = (df['RSI_14']<70)&(df['RSI_14']>30)
mask_le5 = (df['PSARl_0.02_0.2'] < df['close']) & (df['psar_flip_dir']>0)
mask_le6 = (df['RSI_14']<40)
mask_le7 = (df['flag_grad_ema']>=0)

In [21]:
df['ema_crossing_pos'] = 0
df.loc[mask_le1, 'ema_crossing_pos'] = 1
df['macd_pos'] = 0
df.loc[mask_le2, 'macd_pos'] = 1
df['close_to_bbl'] = 0
df.loc[mask_le3, 'close_to_bbl'] = 1
df['rsi_30_to_70'] = 0
df.loc[mask_le4, 'rsi_30_to_70'] = 1
df['PSAR_bellow_close'] = 0
df.loc[mask_le5, 'PSAR_bellow_close'] = 1

In [22]:
df['buy_signal'] = np.nan
# df.loc[(mask_le1 & mask_le4) | (mask_le5 & mask_le4 & mask_le2) | (mask_le2 & mask_le6 & mask_le3), 'long_entry'] = 1
df.loc[(mask_le1 & mask_le4) | (mask_le6 & mask_le7), 'buy_signal'] = 1

In [23]:
# df['long_entry'] = df[['ema_crossing_pos', 'macd_pos', 'rsi_30_to_70', 'PSAR_bellow_close']].sum(axis=1)

# Sell signal

In [24]:
mask_lex1 = (df['ema_short_above_or_below']==-1)&(df['flag_ema_crossing']==1)
mask_lex2 = (df['RSI_14']>55)
mask_lex3 = (df['psar_flip_dir'] == -1)
mask_lex4 = (df['flag_grad_ema']<0)
mask_lex5 = (df['MACDh_12_26_9']<0)

In [25]:
df['ema_crossing_neg'] = 0
df.loc[mask_lex1, 'ema_crossing_neg'] = 1
df['rsi_above_70'] = 0
df.loc[mask_lex2, 'rsi_above_70'] = 1
df['psar_flip_neg'] = 0
df.loc[mask_lex3, 'psar_flip_neg'] = 1
df['macd_neg'] = 0
df.loc[mask_lex5, 'macd_neg'] = 1

In [26]:
df['sell_signal'] = 0
df.loc[(mask_lex1) | (mask_lex2 & mask_lex4), 'sell_signal'] = 1

# Bullish / bearish confirmation

In [27]:
mask_bulber = (df['ADX_14']>=25)
mask_bul = (df['DMP_14']>=25)
mask_ber = (df['DMN_14']>=25)

df['trend_confirm'] = 0
df.loc[mask_bulber & mask_bul, 'trend_confirm'] = 1
df.loc[mask_bulber & mask_ber, 'trend_confirm'] = -1

# Over-sold/buy Marker

In [59]:
mask_os1 = (df['RSI_14']<=20)
mask_os2 = (df['r_close_bbl']<=1.000)
mask_ob1 = (df['RSI_14']>=80)
mask_ob2 = (df['r_close_bbu']>=1.000)

In [60]:
df['oversold_confirm'] = 0
df.loc[mask_os1, 'oversold_confirm'] = 1
df.loc[mask_ob1, 'oversold_confirm'] = -1

# Check data

In [61]:
df.to_clipboard()

In [31]:
df.iloc[-30:]

,date,open,high,low,close,volume,idx_int,EMA_10,EMA_50,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,psar_flip_dir,r_ema_s_m,gradient_ema_10,gradient_rsi_14,gradient_ls,gradient_ema_50,flag_ema_crossing,flag_grad_ema,flag_grad_ema_50,flag_grad_rsi,flag_grad_ls,ema_short_above_or_below,r_close_bbu,r_close_bbl,r_ema_bbu,r_ema_bbl,ema_crossing_pos,macd_pos,close_to_bbl,rsi_30_to_70,PSAR_bellow_close,buy_signal,ema_crossing_neg,rsi_above_70,psar_flip_neg,macd_neg,sell_signal,trend_confirm,oversold_confirm
320,2024-11-09 18:59:00,3049.25,3051.75,3049.23,3050.98,99.0,320,3048.209812,3043.629965,63.540522,3044.455243,3047.4445,3050.433757,0.196181,1.091368,1.264801,0.070091,1.194710,3043.374678,NaN,0.06,0,21.829393,25.317489,15.140091,0,1.001505,0.491775,4.238452,1.685,0.270818,-0.0,1,1,1,1,1,1.000179,1.002143,0.999271,1.001233,0,1,0,1,0,NaN,0,1,0,0,0,0,-1
321,2024-11-09 19:00:00,3050.98,3054.90,3050.72,3053.22,153.0,321,3049.120755,3044.006045,68.015098,3043.997861,3047.8130,3051.628139,0.250353,1.208624,1.586035,0.313060,1.272975,3043.877197,NaN,0.08,0,23.178525,31.498523,13.270010,0,1.001680,0.763270,4.270189,1.985,0.338041,-0.0,1,1,1,1,1,1.000522,1.003030,0.999178,1.001683,0,1,0,1,0,NaN,0,1,0,0,0,0,-1
322,2024-11-09 19:01:00,3053.14,3086.13,3049.81,3080.87,1304.0,322,3054.893345,3045.451690,87.845133,3034.596634,3049.4900,3064.383366,0.976778,1.553489,4.025338,2.201890,1.823448,3044.759021,NaN,0.10,0,27.350689,60.711112,6.155465,0,1.003100,3.341767,12.152305,14.945,0.910863,-0.0,1,1,1,1,1,1.005380,1.015249,0.996903,1.006688,0,1,0,0,0,NaN,0,1,0,0,0,1,-1
323,2024-11-09 19:02:00,3080.87,3082.03,3070.63,3081.37,574.0,323,3059.707283,3046.860251,87.990135,3030.936090,3051.2225,3071.508910,1.329723,1.243047,5.930486,3.285631,2.644855,3048.896119,NaN,0.10,0,31.224840,51.396711,5.211083,0,1.004216,5.293264,9.987518,14.075,1.427103,-0.0,1,1,1,1,1,1.003211,1.016640,0.996158,1.009493,0,1,0,0,0,NaN,0,1,0,0,0,1,-1
324,2024-11-09 19:03:00,3080.98,3083.87,3072.90,3078.71,313.0,324,3063.162322,3048.109261,82.360974,3029.253055,3052.7420,3076.230945,1.538875,1.052771,7.143346,3.598792,3.544554,3049.810000,NaN,0.10,0,34.881433,46.647009,4.496223,0,1.004938,4.134488,-2.742079,-1.080,1.328785,-0.0,1,1,-1,-1,1,1.000806,1.016326,0.995752,1.011194,0,1,0,0,0,NaN,0,1,0,0,0,1,-1
325,2024-11-09 19:04:00,3078.32,3079.15,3071.62,3073.79,152.0,325,3065.094627,3049.116349,73.051846,3029.078258,3054.0945,3079.110742,1.638210,0.893654,7.619708,3.260123,4.359584,3053.442000,NaN,0.10,0,37.851992,42.352216,5.647324,0,1.005240,2.693672,-7.469145,-3.790,1.128049,-0.0,1,1,-1,-1,1,0.998272,1.014761,0.995448,1.011890,0,1,0,0,0,NaN,0,1,0,0,0,1,0
326,2024-11-09 19:05:00,3073.34,3073.72,3066.87,3073.71,237.0,326,3066.661059,3050.080806,72.907544,3029.250133,3055.4315,3081.612867,1.713759,0.849075,7.899710,2.832100,5.067610,3056.710800,NaN,0.10,0,39.157221,38.815357,10.908016,0,1.005436,1.749368,-4.726715,-2.500,0.985772,-0.0,1,1,-1,-1,1,0.997435,1.014677,0.995148,1.012350,0,1,0,0,0,NaN,0,1,0,0,0,1,0
327,2024-11-09 19:06:00,3073.71,3077.67,3072.22,3077.01,105.0,327,3068.542684,3051.136853,75.093123,3029.273437,3056.8570,3084.440563,1.804701,0.865308,8.292307,2.579758,5.712549,3059.652720,NaN,0.10,0,40.662492,41.041907,10.186507,0,1.005705,1.724029,1.020638,1.610,1.010252,-0.0,1,1,1,1,1,0.997591,1.015758,0.994846,1.012963,0,1,0,0,0,NaN,0,1,0,0,0,1,0
328,2024-11-09 19:07:00,3077.25,3077.59,3074.06,3075.46,144.0,328,3069.800378,3052.090702,72.149034,3029.634037,3058.1475,3086.660963,1.864754,0.803585,8.381752,2.135362,6.246390,3062.300448,NaN,0.10,0,42.060245,39.231827,9.737249,0,1.005802,1.569660,-0.379255,0.875,1.004948,-0.0,1,1,-1,1,1,0.996371,1.015126,0.994538,1.013258,0,1,0,0,0,NaN,0,1,0,0,0,1,0
329,2024-11-09 19:08:00,3075.14,3075.54,3070.16,3070.51,102.0,329,3069.929400,3052.813027,63.576563,3030.799469,3059.3205,3087.841531,1.864534